# Create a Ryu OpenFlow Controller

#### Configure the Environment

In [1]:
# Set project and region.
export OS_REGION_NAME=CHI@UC
export OS_PROJECT_NAME=CH-822154

# Define networking variables.
NODE_TYPE=compute_haswell
KEYPAIR_NAME="my_chameleon_key"
KEYPAIR_LOCATION="/home/"${USER}"/work/"${KEYPAIR_NAME}".pem" # 
LEASE_NAME="${USER}RyuControllerLease"
SERVER_NAME="${USER}RyuController"

#### Create Lease

In [2]:
# Obtain public network id.
PUBLIC_NETWORK_ID=$(openstack network show public -f value -c id)

# Reserve a node and a floating IP on public network.
blazar lease-create \
  --physical-reservation min=1,max=1,resource_properties="[\"=\", \"\$node_type\", \"$NODE_TYPE\"]" \
  --reservation "resource_type=virtual:floatingip,network_id=$PUBLIC_NETWORK_ID,amount=1" \
  --start-date "$(date +'%Y-%m-%d %H:%M' -d'+1 minute')" \
  --end-date "$(date +'%Y-%m-%d %H:%M' -d'+1 day')" \
  "$LEASE_NAME"

Matches: ('min=1,max=1', 'resource_properties', '["=", "$node_type", "compute_haswell"]')
Matches: ('min=1', 'max', '1')
Matches: (None, 'min', '1')
Matches: ('resource_type=virtual:floatingip,network_id=44b38c44-2a42-4b6d-b129-6c8f1b2a1375', 'amount', '1')
Matches: ('resource_type=virtual:floatingip', 'network_id', '44b38c44-2a42-4b6d-b129-6c8f1b2a1375')
Matches: (None, 'resource_type', 'virtual:floatingip')
Created a new lease:
+--------------+----------------------------------------------------------------------------+
| Field        | Value                                                                      |
+--------------+----------------------------------------------------------------------------+
| created_at   | 2021-03-09 23:40:52                                                        |
| degraded     | False                                                                      |
| end_date     | 2021-03-10T23:40:00.000000                                                 |
| 

#### Start the Server

In [4]:
# Create a new server using your reservation
openstack server create --max-width 80 \
                        --key-name $KEYPAIR_NAME \
                        $(lease_server_create_default_args "$LEASE_NAME") \
                        $SERVER_NAME

+-------------------------------------+----------------------------------------+
| Field                               | Value                                  |
+-------------------------------------+----------------------------------------+
| OS-DCF:diskConfig                   | MANUAL                                 |
| OS-EXT-AZ:availability_zone         |                                        |
| OS-EXT-SRV-ATTR:host                | None                                   |
| OS-EXT-SRV-ATTR:hypervisor_hostname | None                                   |
| OS-EXT-SRV-ATTR:instance_name       |                                        |
| OS-EXT-STS:power_state              | NOSTATE                                |
| OS-EXT-STS:task_state               | scheduling                             |
| OS-EXT-STS:vm_state                 | building                               |
| OS-SRV-USG:launched_at              | None                                   |
| OS-SRV-USG:terminated_at  

#### Associate the Floating IP

In [5]:
# Find the unused floating IP you reserved
FLOATING_IP=$(lease_list_floating_ips "$LEASE_NAME" | head -n1)

# Add your floating IP to your server
openstack server add floating ip "$SERVER_NAME" "$FLOATING_IP" \
  && echo "Attached floating ip $FLOATING_IP!"

Attached floating ip 192.5.87.223!


#### Run Script

This script observes your node using TCP access.

In [6]:
SCRIPT_FILE='script.sh'

cat >$SCRIPT_FILE <<EOF
#!/bin/bash
            {
            set -e -u -o pipefail

            RECIPE_REPO="https://github.com/RENCI-NRIG/exogeni-recipes.git"
            RECIPE_DIR="/opt/exogeni-recipes"
            RECIPE_APP="openflow-controller/docker"
            DOCKER_IMAGE="centos-ryu"
            DOCKER_CONTAINER_NAME="ryu-controller"
            OFP_TCP_LISTEN_PORT="6653"
            RYU_APP="/opt/ryu_app/simple_switch_13_custom_chameleon.py"
            MIRROR_PORT="10000"

            yum install -y yum-utils device-mapper-persistent-data lvm2 vim

            echo "Installing Docker ..."
            yum-config-manager --add-repo https://download.docker.com/linux/centos/docker-ce.repo
            yum install -y docker-ce
            systemctl start docker

            echo "Configuring Ryu controller ..."
            git clone  --no-checkout \$RECIPE_REPO \$RECIPE_DIR
            cd \$RECIPE_DIR && git config core.sparsecheckout true
            echo "\$RECIPE_APP/*" >> .git/info/sparse-checkout
            git read-tree -m -u HEAD
            pushd \${RECIPE_DIR}/\${RECIPE_APP}
            sed -r -i 's/^(RYU_APP=.*)/#\1/g' ryu_start.sh
            sed -r -i 's/^(OFP_TCP_LISTEN_PORT=.*)/#\1/g' ryu_start.sh

            echo "Building Ryu controller image ..."
            docker build -t \${DOCKER_IMAGE} .

            echo "Starting Ryu controller ..."
            docker run --rm -dit \
              -p \$OFP_TCP_LISTEN_PORT:\$OFP_TCP_LISTEN_PORT -p 8080:8080 \
              -v opt_ryu_chameleon:/opt/ryu_chameleon \
              -v opt_ryu:/opt/ryu \
              -v var_log_ryu:/var/log/ryu \
              -v var_run_ryu:/var/run/ryu \
              -e RYU_APP=\$RYU_APP -e OFP_TCP_LISTEN_PORT=\$OFP_TCP_LISTEN_PORT \
              --name=\$DOCKER_CONTAINER_NAME \
              \$DOCKER_IMAGE

            echo "Opening OF controller port ..."
            ufw allow \$OFP_TCP_LISTEN_PORT

            echo "Done."
            } > /tmp/boot.log 2>&1
EOF

In [7]:
#Copy the host file to the server
scp -o "UserKnownHostsFile=/dev/null" \
    -o "StrictHostKeyChecking=no" \
    -i $KEYPAIR_LOCATION \
    $SCRIPT_FILE cc@${FLOATING_IP}:.

#Append the file to the server /etc/host file
ssh -t \
    -o "UserKnownHostsFile=/dev/null" \
    -o "StrictHostKeyChecking=no" \
    -i $KEYPAIR_LOCATION \
    cc@${FLOATING_IP} \
    "sudo sh -c 'chmod +x $SCRIPT_FILE && ./$SCRIPT_FILE'"

script.sh                                     100% 1940     1.2MB/s   00:00    
Connection to 192.5.87.223 closed.


#### Clean up

In [ ]:
openstack server delete $SERVER_NAME
blazar lease-delete $LEASE_NAME